In [1]:
import sys
sys.path.append('..')

import numpy as np
import cirq
import mitiq
from cirq.contrib.qasm_import import circuit_from_qasm
import matplotlib.pyplot as plt
from functools import partial
from tqdm import tqdm
import sympy as sp


import mosaic



**construct the ideal circuit**

In [2]:
qubits = cirq.LineQubit.range(2)
circ = cirq.Circuit([
    cirq.H(qubits[0]),
    cirq.H(qubits[1]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.X.on(qubits[0]),
    cirq.T.on(qubits[1]),
    cirq.CNOT(qubits[1], qubits[0]),
    cirq.ZPowGate(exponent=0.1).on(qubits[0]),
    cirq.S.on(qubits[1]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.XPowGate(exponent=0.1).on(qubits[0]),
    cirq.H.on(qubits[1]),
    cirq.CNOT(qubits[1], qubits[0]),
])
# circ = cirq.Circuit([  # only involves 2-qubit gates
#     cirq.XXPowGate(exponent=0.1).on(qubits[0], qubits[1]),
#     cirq.Rz(rads=0.2).on(qubits[1]).controlled_by(qubits[0]),
#     cirq.YYPowGate(exponent=0.1).on(qubits[0], qubits[1]),
#     cirq.Rx(rads=0.1).on(qubits[0]).controlled_by(qubits[1]),
#     cirq.ZZPowGate(exponent=0.1).on(qubits[0], qubits[1]),
# ])
circ

In [3]:
init_state = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]).astype(complex)
final_state = circ.unitary() @ init_state @ circ.unitary().conj().T
prob_11 = final_state[1, 1].real
prob_11

In [4]:
final_state.round(3)

In [5]:
n = len(qubits)
p_cirq = 0.01
p = 4**n / (4**n - 1) * p_cirq
print(f'p_cirq = {p_cirq}, p = {p}')

In [6]:
circ_noise_interleaved = cirq.Circuit()
for g in circ.all_operations():
    circ_noise_interleaved.append(g)
    circ_noise_interleaved.append(cirq.depolarize(p_cirq, 2).on(*qubits))
circ_noise_interleaved

In [7]:
r = mosaic.circuits.num_gates(circ)
p_r = 1 - (1 - p)**r
p_r_cirq = p_r * (4**n - 1) / 4**n
print(f'p_r_cirq = {p_r_cirq}, p = {p_r}')
circ_noise = circ.copy()
circ_noise.append(cirq.depolarize(p_r_cirq, 2).on(*qubits))
circ_noise

**noisy simulation comparison**

In [8]:
sim = cirq.DensityMatrixSimulator()

rho1 = sim.simulate(circ_noise_interleaved).final_density_matrix
rho2 = sim.simulate(circ_noise).final_density_matrix

assert np.allclose(rho1, rho2, atol=1e-5), "rho1 and rho2 should be equal!"

**append inverse channel to `circ_noise`**

In [9]:
circ_noise

In [10]:
final_state_noise = cirq.DensityMatrixSimulator().simulate(circ_noise).final_density_matrix
prob_11_noise = final_state_noise[1, 1].real
print('disparity = ', prob_11_noise - prob_11)

**Testing our error mitigation approach**:

- baseline: conventional PEC method
- baseline: linear combination of two noisy channels: $\mathcal{E}^{-1} = \eta_1\mathcal{E}_1 - \eta_2\mathcal{E}_2$ in which both $\mathcal{E}_1$ and $\mathcal{E}_2$ are depolarizing channels (CPTP)
- our end-to-end PEC method: ...

In [11]:
# baseline: conventional PEC

# def execute(circ: cirq.Circuit):
#     return cirq.DensityMatrixSimulator().simulate(circ).final_density_matrix[1,1].real

def execute_noisy(circ: cirq.Circuit, p_cirq):
    n = cirq.num_qubits(circ)
    p = 4**n / (4**n - 1) * p_cirq
    p_r = 1 - (1 - p)**r
    p_r_cirq = p_r * (4**n - 1) / 4**n
    qubits = list(circ.all_qubits())
    circ_noise = circ.copy()
    circ_noise.append(cirq.depolarize(p_r_cirq, 2).on(*qubits))
    return cirq.DensityMatrixSimulator().simulate(circ_noise).final_density_matrix[1,1].real

def execute_noise_interleaved(circ: cirq.Circuit, p_cirq):
    circ_noise_interleaved = cirq.Circuit()
    for moment in circ:
        circ_noise_interleaved.append(moment)
        circ_noise_interleaved.append(cirq.depolarize(p_cirq, 2).on(*qubits))
    return cirq.DensityMatrixSimulator().simulate(circ_noise_interleaved).final_density_matrix[1,1].real


In [12]:
rates_cirq = np.linspace(0, 0.1, 11)
noisy_vals = [execute_noisy(circ, p_cirq) for p_cirq in rates_cirq]

In [202]:
reps = mitiq.pec.represent_operations_in_circuit_with_global_depolarizing_noise(circ, p)
mitiq.pec.execute_with_pec(circ, partial(execute_noisy, p_cirq=p_cirq), representations=reps, random_state=42)

baseline: conventional PEC method

In [211]:
pec_vals = []

for p_cirq in tqdm(rates_cirq):
    reps = mitiq.pec.represent_operations_in_circuit_with_global_depolarizing_noise(circ, p_cirq)
    pec_vals.append(mitiq.pec.execute_with_pec(circ, partial(execute_noisy, p_cirq=p_cirq), 
                                                  representations=reps, random_state=42))

In [214]:
plt.figure(figsize=(5, 3))
plt.plot(rates_cirq, noisy_vals, 'o-', label='Noisy')
plt.plot(rates_cirq, pec_vals, 'o-', label='PEC')
plt.axhline(prob_11, color='k', linestyle='--', label='Ideal')
plt.legend()
plt.show()

In [150]:
# def execute_local_depolarizing(circ: cirq.Circuit, p_cirq):
#     circ_local_noise = circ.copy()
#     circ_local_noise = circ_local_noise.with_noise(cirq.depolarize(p_cirq))
#     return cirq.DensityMatrixSimulator().simulate(circ_local_noise).final_density_matrix[1,1].real

# noisy_vals = [execute_local_depolarizing(circ, p_cirq) for p_cirq in rates_cirq[:6]]

# pec_vals = []

# for p_cirq in tqdm(rates_cirq):
#     p = 4**n / (4**n - 1) * p_cirq
#     reps = mitiq.pec.represent_operations_in_circuit_with_local_depolarizing_noise(circ, p)
#     pec_vals.append(mitiq.pec.execute_with_pec(circ, 
#                                                partial(execute_local_depolarizing, p_cirq=p_cirq), 
#                                                representations=reps, random_state=42))


baseline: linear combination of two noisy channels: $\mathcal{E}^{-1} = \eta_1\mathcal{E}_1 - \eta_2\mathcal{E}_2$ in which both $\mathcal{E}_1$ and $\mathcal{E}_2$ are depolarizing channels (CPTP)

In [13]:
def inv_channel_rate(epsilon, n=1):
    return (4**n-1) * epsilon / (4**n-1 - 4**n * epsilon)

def decompose_inv_channel(eta_1, epsilon_inv):
    """
    The result is a relationship between epsilon_1 and epsilon_2 for subsequent manual settings for their specific values.
    """
    import sympy as sp
    # 定义未知数
    epsilon_1, epsilon_2 = sp.symbols('epsilon_1 epsilon_2')

    # 已知常数
    eta_2 = eta_1 - 1  # 例如

    # 定义方程
    eq1 = sp.Eq(eta_1 * (1 - epsilon_1) - eta_2 * (1 - epsilon_2), 1 + epsilon_inv)
    eq2 = sp.Eq(eta_1 * epsilon_1 + epsilon_inv, eta_2 * epsilon_2)

    # 求解方程组
    solutions = sp.solve((eq1, eq2), (epsilon_1, epsilon_2))

    return solutions

In [94]:
eta_1 = r // 2 # it is a heuristic super-parameter
eta_2 = eta_1 - 1

epsilon_1 = sp.symbols('epsilon_1')
epsilon_2 = sp.symbols('epsilon_2')
sol = decompose_inv_channel(eta_1, round(inv_channel_rate(p_r_cirq, 2), 10))
sol

In [95]:
p2 = 0.05
p1 = float(sol[epsilon_1].subs(epsilon_2, p2))
print(f"p1 = {p1}, p2 = {p2}")

In [96]:
# verify the correctness of (eta_1, eta_2, p1, p2) -> K_inv
assert np.allclose(eta_1 * (1 - p1) - eta_2 * (1 - p2), 1 + inv_channel_rate(p_r_cirq, 2))
assert np.allclose(eta_1 * p1 - eta_2 * p2, - inv_channel_rate(p_r_cirq, 2))

channel = cirq.depolarize(p_r_cirq, 2)
super_opr = cirq.kraus_to_superoperator(cirq.protocols.kraus_protocol.kraus(channel))
super_opr_inv = np.linalg.inv(super_opr)

super_opr_1 = cirq.kraus_to_superoperator(cirq.protocols.kraus_protocol.kraus(cirq.depolarize(p1, 2)))
super_opr_2 = cirq.kraus_to_superoperator(cirq.protocols.kraus_protocol.kraus(cirq.depolarize(p2, 2)))

K_inv = (eta_1 * super_opr_1 - eta_2 * super_opr_2)

assert np.allclose(super_opr_inv, K_inv)

In [97]:
# let's do linear combination of the effect of inverse channel
circ_noise

In [98]:
circ_noise_1 = circ_noise.copy()
circ_noise_2 = circ_noise.copy()
circ_noise_1.append(cirq.depolarize(p1, 2).on(*qubits))
circ_noise_2.append(cirq.depolarize(p2, 2).on(*qubits))

In [99]:
res1 = cirq.DensityMatrixSimulator().simulate(circ_noise_1).final_density_matrix[1,1].real
res2 = cirq.DensityMatrixSimulator().simulate(circ_noise_2).final_density_matrix[1,1].real
res1, res2

In [100]:
res = eta_1 * res1 - eta_2 * res2
print(prob_11, prob_11_noise, res)
print('disparity = ', res - prob_11)

In [102]:
rates_cirq_inv = np.vectorize(inv_channel_rate)(rates_cirq)
print(rates_cirq)
print(rates_cirq_inv.round(4))

In [134]:
def extract_slope(f: sp.Add):
    return f.as_coefficients_dict()[sp.symbols('epsilon_1')]

In [155]:
er_vals = []
for p_cirq in rates_cirq:
    print('---')
    n = cirq.num_qubits(circ)
    p = 4**n / (4**n - 1) * p_cirq
    p_r = 1 - (1 - p)**r
    p_r_cirq = p_r * (4**n - 1) / 4**n
    qubits = list(circ.all_qubits())
    circ_er = circ.copy() # error-renormalizing circuit
    circ_er.append(cirq.depolarize(p_r_cirq, 2).on(*qubits))
    p_r_cirq_inv = inv_channel_rate(p_r_cirq, 2)
    # print(p_r_cirq, p_r_cirq_inv)

    eta_1 = r # it is a heuristic super-parameter
    eta_2 = eta_1 - 1

    epsilon_1 = sp.symbols('epsilon_1')
    epsilon_2 = sp.symbols('epsilon_2')
    sol = decompose_inv_channel(eta_1, round(inv_channel_rate(p_r_cirq, 2), 10))
    # print(sol)
    # extract "k" from "kx + b"
    p2 = p_r_cirq / 2 * float(sol[epsilon_1].as_coefficients_dict()[epsilon_2])
    p1 = float(sol[epsilon_1].subs(epsilon_2, p2))
    print(f"p1 = {p1}, p2 = {p2}")

    circ_noise_1 = circ_er.copy()
    circ_noise_2 = circ_er.copy()
    circ_noise_1.append(cirq.depolarize(p1, 2).on(*qubits))
    circ_noise_2.append(cirq.depolarize(p2, 2).on(*qubits))
    res1 = cirq.DensityMatrixSimulator().simulate(circ_noise_1).final_density_matrix[1,1].real
    res2 = cirq.DensityMatrixSimulator().simulate(circ_noise_2).final_density_matrix[1,1].real
    res = eta_1 * res1 - eta_2 * res2
    er_vals.append(res)


Tips:
- 增加 $\eta_1$, $eta_2$ 能够减小 $\epsilon_1$ 和 $\epsilon_2$
- 给定 $\eta_1$, $eta_2$ 的情况下，求解出的最终结果是 $\epsilon_1 = k_{\eta_1, \eta_2}\epsilon - b_{\epsilon}$. 其中 $k_{\eta_1, \eta_2}$ only depends on $(\eta_1,\eta_2)$; $b_\epsilon > 0$ 且与 $\epsilon$ 正相关. 因此对于更大的noise level, 可以考虑增大 $\eta_1$, $\eta_2$ 以减小 $\epsilon_1$ 和 $\epsilon_2$

In [157]:
plt.figure(figsize=(5, 3))
plt.plot(rates_cirq, noisy_vals, 'o-', label='Noisy')
plt.plot(rates_cirq, er_vals, 'o-', label='Error renormalizing')
plt.axhline(prob_11, color='k', linestyle='--', label='Ideal')
plt.legend()
plt.show()

our end-to-end PEC method for general Pauli channel $\mathcal{E}$: 